In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re 
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from textblob import TextBlob
from nltk.util import ngrams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

data = pd.read_csv('comments_500k_201904.csv')

In [3]:
data.groupby('subreddit').agg('count').sort_values('body', ascending=False)


,parent_id,subreddit_id,created_utc,author,score,body
subreddit,,,,,,
AskReddit,24772,24772,24772,24772,24772,24772
SquaredCircle,7355,7355,7355,7355,7355,7355
politics,6316,6316,6316,6316,6316,6316
hockey,5659,5659,5659,5659,5659,5659
teenagers,5421,5421,5421,5421,5421,5421
nba,5281,5281,5281,5281,5281,5281
memes,4426,4426,4426,4426,4426,4426
dankmemes,4327,4327,4327,4327,4327,4327
nfl,3499,3499,3499,3499,3499,3499


In [4]:
# score analysis 

In [6]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values().head(20))

,score
subreddit,
youdontsurf,-26.000000
Ashe_Maree,-18.000000
TwoRedditorsOneCup,-14.750000
wendystwitter,-14.000000
Ethan,-12.000000
SheLikesItRough,-11.333333
TransSpace,-11.250000
WhereIsAssange,-10.000000
ZahraElise,-9.000000


In [5]:
pd.DataFrame(data.groupby('subreddit')['score'].agg('mean').sort_values(ascending=False).head(20))

,score
subreddit,
BarbaraWalters4Scale,250.500000
Emuwarflashbacks,167.000000
IAmA,160.442177
RectalStickers,156.000000
explainitkakyoin,126.000000
AMADisasters,121.000000
HumanTippyTaps,111.000000
oversharing,106.333333
AdrenalinePorn,93.000000


### Vader analysis 

In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
#     print("{}".format(str(score)))
    return score
top_20 = data.sort_values('subreddit').reset_index()
top_20['senti']= data['body'].map(lambda t: sentiment_analyzer_scores(str(t))) 
top_20['senti_neu'] = top_20.senti.apply(lambda x: x.get('neu'))
top_20['senti_pos'] = top_20.senti.apply(lambda x: x.get('pos'))
top_20['senti_neg'] = top_20.senti.apply(lambda x: x.get('neg'))

In [10]:
pd.DataFrame(top_20.groupby('subreddit')['senti_pos'].agg('mean').sort_values(ascending=False).head(20))

,senti_pos
subreddit,
pnwriders,1.0
afterlife,1.0
NicoleKopchak,1.0
StairsintheWoods,1.0
Missing411,1.0
RnBHeads,1.0
naturepics,1.0
u_3062fran,1.0
LucioRollouts,1.0


In [12]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neg'].agg('mean').sort_values(ascending=False).head(50))

,senti_neg
subreddit,
mmmp,1.000
XevTainton,1.000
itslenny,1.000
Danii_Banks,1.000
gachimuchi,1.000
gamers,1.000
Ultraman,1.000
cardcaptorsakura,1.000
pern,1.000


In [17]:
pd.DataFrame(top_20.groupby('subreddit')['senti_neu'].agg('mean'). \
                                         sort_values(ascending=False).head(20))

,senti_neu
subreddit,
07Scape,1.0
NessMains,1.0
montypythongifs,1.0
NewOrleansBeer,1.0
NewGirl,1.0
NewDads,1.0
mordheim,1.0
NetsuiteAdmin,1.0
Netsuite,1.0


In [22]:
pd.DataFrame(top_20[['subreddit','senti_neg', 'senti_neu', 'senti_pos']]).sample(20)

,subreddit,senti_neg,senti_neu,senti_pos
468060,tifu,0.000,1.000,0.000
162038,MortalKombat,0.000,1.000,0.000
187253,Professors,0.000,0.915,0.085
202531,Sekiro,0.000,1.000,0.000
103714,FanFiction,0.000,1.000,0.000
397030,newjersey,0.000,1.000,0.000
209530,Sneakers,0.000,0.920,0.080
257612,anime,0.000,1.000,0.000
378704,memes,0.315,0.180,0.505
145306,LivestreamFail,0.196,0.804,0.000


In [23]:
data.groupby('subreddit').agg('count').sort_values('body', ascending=False)

,parent_id,subreddit_id,created_utc,author,score,body
subreddit,,,,,,
AskReddit,24772,24772,24772,24772,24772,24772
SquaredCircle,7355,7355,7355,7355,7355,7355
politics,6316,6316,6316,6316,6316,6316
hockey,5659,5659,5659,5659,5659,5659
teenagers,5421,5421,5421,5421,5421,5421
nba,5281,5281,5281,5281,5281,5281
memes,4426,4426,4426,4426,4426,4426
dankmemes,4327,4327,4327,4327,4327,4327
nfl,3499,3499,3499,3499,3499,3499


In [30]:
nlp = spacy.load('en_core_web_sm')


In [35]:
# group = (data.groupby('subreddit')['body'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())
# group['words'] = group['body'].map(lambda t:[token.text for token in nlp(t) if token.is_stop != True \
#                                              and token.is_punct != True] )
# group['noun_words'] = group['body'].map(lambda t: [token.text for token in nlp(t) \
#                                               if token.is_stop != True and token.is_punct != True \
#                                               and token.pos_ == "PROPN"])
# group['count'] = group['words'].map(lambda t: Counter(t).most_common(10))
# group['n_count'] = group['noun_words'].map(lambda t: Counter(t).most_common(10))


In [34]:
group.head()

,subreddit,body
0,07Scape,Don't pollute this sub with your religious pro...
1,1000ccplus,"��veins��,For the love of fuck"
2,1000daysofpractice,Sounds like you kicked butt today! Good job ��...
3,100movies365days,When you consider that Vince Vaughn has “A Lis...
4,100thieves,"[deleted],Can fakegod play mid?? lol,Very true..."


In [ ]:
myList=group[group.subreddit=='CollegeBasketball']['count']
for p in myList: 
    print(pd.DataFrame(p))